In [ ]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import random

In [ ]:
genres= ['%E5%89%A7%E6%83%85', '%E5%96%9C%E5%89%A7', '%E5%8A%A8%E4%BD%9C', '%E7%88%B1%E6%83%85', '%E7%A7%91%E5%B9%BB', '%E5%8A%A8%E7%94%BB',
            '%E6%82%AC%E7%96%91', '%E6%83%8A%E6%82%9A', '%E6%81%90%E6%80%96', '%E7%8A%AF%E7%BD%AA', '%E5%90%8C%E6%80%A7', '%E9%9F%B3%E4%B9%90',
            '%E6%AD%8C%E8%88%9E', '%E4%BC%A0%E8%AE%B0', '%E5%8E%86%E5%8F%B2', '%E6%88%98%E4%BA%89', '%E8%A5%BF%E9%83%A8', '%E5%A5%87%E5%B9%BB',
            '%E5%86%92%E9%99%A9', '%E7%81%BE%E9%9A%BE', '%E6%AD%A6%E4%BE%A0', '%E6%83%85%E8%89%B2']
proxies = {'http': 'http://117.88.176.196:3000',
            'http': 'http://111.222.141.127:8118'}

In [ ]:
def check_none(string):
    if string == None:
        return ''
    else:
        return string

In [ ]:
def request_douban(url):
    try:
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        response = requests.get(url,headers=headers,proxies=proxies)
        if response.status_code == 200:
            return response.text
    except requests.RequestException:
        print('requesting fails')
        return None

In [ ]:
def get_url_list(text):
    '''
    获取一页20个电影的url
    '''
    dic = json.loads(text)
    url_list = [dic['url'] for dic in dic['data']]
    return url_list

In [ ]:
def get_multi_info(soup):
    '''
    处理含多项的内容
    '''
    
    info = ''
    for item in soup:
        info = ('' if info == '' else '/').join((info, item.string))
    return info

In [ ]:
def get_info(url):
    '''
    电影详细信息
    '''

    html = request_douban(url)
    if html == None:
        print('未获取到电影url')
    soup = BeautifulSoup(html, 'lxml')

    pattern = re.compile(r'\d+')
    id = pattern.findall(url)[0]
    
    title = soup.find('span', property='v:itemreviewed').string
    title = check_none(title)

    soup_div = soup.find(id='info')

    try:
        director = get_multi_info(soup_div.find_all('span')[0].find_all('a'))
    except AttributeError:
        director = ''
    director = check_none(director)

    try:
        editor = get_multi_info(soup_div.find_all('span')[3].find_all('a'))
    except AttributeError:
        editor = ''
    editor = check_none(editor)

    try:
        type_ = get_multi_info(soup_div.find_all('span',property='v:genre'))
    except AttributeError:
        type_ = ''
    type_ = check_none(type_)

    try:
        region = soup_div.find(text='制片国家/地区:').next_element.lstrip().rstrip()
    except AttributeError:
        region = ''
    region = check_none(region)

    try:
        languages = soup_div.find(text='语言:').next_element.lstrip().rstrip()
    except AttributeError:
        languages = ''
    languages = check_none(languages)
    
    try:
        release_date = soup_div.find('span', property='v:initialReleaseDate').string
    except AttributeError:
        release_date = ''
    release_date = check_none(release_date)

    try:
        mins = soup_div.find('span', property='v:runtime').string
    except AttributeError:
        mins = ''
    mins = check_none(mins)

    try:
        alias = soup_div.find(text='又名:').next_element.lstrip().rstrip()
    except AttributeError:
        alias = ''
    alias = check_none(alias)

    try:
        rating = soup.find('strong', property='v:average').string
    except AttributeError:
        rating = ''
    rating = check_none(rating)

    try:
        douban_votes = soup.find('span', property='v:votes').string
    except AttributeError:
        douban_votes = ''
    douban_votes = check_none(douban_votes)

    try:
        storyline = soup.find('span', property='v:summary').text
    except AttributeError:
        storyline = ''
    storyline = check_none(storyline)
    storyline = ''.join(storyline.split())

    url_cele = url + 'celebrities'
    html_cele = request_douban(url_cele)
    soup_cele = BeautifulSoup(html_cele, 'lxml')

    actors=''
    try:
        actors_list = soup_cele.find_all(class_='list-wrapper')[1].find_all('span',class_='name')
    except IndexError:
        actors_list = soup_cele.find_all(class_='list-wrapper')[0].find_all('span',class_='name')
    for item in actors_list:
        actor = item.find('a').string
        if actor:
            actors = ('' if actors == '' else '/').join((actors, item.find('a').string))

    print(id+' | '+title+' | '+director+' | '+editor+' | '+type_+' | '+region+' | '+languages+' | '+release_date+' | '+mins+' | '+alias+' | '+
            rating+' | '+douban_votes+' | '+actors+' | '+storyline)

    info = (id,title,director,editor,type_,region,languages,release_date,mins,alias,rating,douban_votes,actors,storyline)
    return info

In [ ]:
def main():
    # 按类型爬，先每个类型前20个，再20~40个，以此类推
    i = 0
    for page in range(400):
        for genre in genres:
            url = 'https://movie.douban.com/j/new_search_subjects?sort=U&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=' + str(page*20) + '&genres=' + genre
            text = request_douban(url)
            if text == None:
                print('没了')
                break
            url_list = get_url_list(text)
            # 写入 .csv
            csv_file = open("douban_movies.csv",'a',encoding='utf-8')
            for url in url_list:
                time.sleep(random.random())
                info = get_info(url)
                csv_file.write(','.join(info))
                csv_file.write("\n")
                i += 1
                print('共','{:d}'.format(i),'行')
            csv_file.close
    return

In [ ]:
main()